지역 이미지를 넣어서 해당 이미지가 어느 지역인지를 추론하는 모델을 활용한 fast API

In [6]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel
from googletrans import Translator

# 모델과 프로세서 로드
model = CLIPModel.from_pretrained("geolocal/StreetCLIP")
processor = CLIPProcessor.from_pretrained("geolocal/StreetCLIP")

# 이미지 불러오기
url = "https://huggingface.co/geolocal/StreetCLIP/resolve/main/sanfrancisco.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

# 지역 이름 입력받기
choices = []
original_choices = []  # 원래 입력한 지역 이름을 저장할 리스트
translator = Translator()  # 번역기 초기화

while True:
    print("지역 이름을 입력하세요. 입력을 끝내려면 'x'를 입력하세요.")
    
    while True:
        location = input("지역 이름: ")
        if location.lower() == "x":
            break
        if location:  # 빈 입력이 아닌 경우만 추가
            translated_location = translator.translate(location, src='ko', dest='en').text
            choices.append(translated_location)  # 번역된 지역 이름 추가
            original_choices.append(location)  # 원래 지역 이름 추가

    # 지역 이름이 입력되지 않았을 경우 재입력 요청
    if not choices:
        print("입력된 지역이 없습니다. 정확한 추론을 위해 한 가지 이상의 지역을 입력해주세요.")
    else:
        break

# 이미지와 입력한 텍스트를 모델에 전달
inputs = processor(text=choices, images=image, return_tensors="pt", padding=True)

# 모델 추론
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
probs = logits_per_image.softmax(dim=1)

# logits_per_image에서 가장 큰 값의 인덱스 구하기
max_index = torch.argmax(logits_per_image).item()

# 가장 높은 유사도를 가진 지역과 유사도 값
most_similar_location = original_choices[max_index]  # 원래 지역 이름 사용
similarity = probs[0, max_index].item() * 100  # 유사도 값을 퍼센트로 변환

# 결과 출력
if similarity < 70:
    print(f"입력하신 지역들은 이미지와 유사도가 낮아 정확하게 찾아줄 수는 없습니다.\n 그러나 이 이미지는 현재 입력하신 지역 중 {most_similar_location}가 가장 유사도가 높습니다. 유사도는 {similarity:.2f}%입니다.")
else:
    print(f"이 이미지는 {most_similar_location}가 가장 유사도가 높습니다. 유사도는 {similarity:.2f}%입니다.")


지역 이름을 입력하세요. 입력을 끝내려면 'x'를 입력하세요.
입력하신 지역들은 이미지와 유사도가 낮아 정확하게 찾아줄 수는 없습니다.
 그러나 이 이미지는 현재 입력하신 지역 중 파리가 가장 유사도가 높습니다. 유사도는 51.05%입니다.
